In [1]:
import pandas as pd

#trip_code: consecutive trips within a day is assigned same trip_code ( else different trip code)
# It is used for merging trips i.e. merge consecutive tripcount as a single output row 
def assign_trip_code(df_single_ap):
    df_single_ap['trip_code']=None
    df_single_ap = df_single_ap.reset_index(drop=True)
    date_old = 0
    trip_count_old = 0
    trip_code = 0
    
    for idx,row in df_single_ap.iterrows():
        #print(idx)
        cur_trip_count = df_single_ap.iloc[idx].trip_prevs
        cur_date = df_single_ap.iloc[idx].date
        #print(cur_trip_count, trip_count_old, trip_code)
        
        if (trip_count_old ) == cur_trip_count:
            df_single_ap.at[idx,'trip_code'] = str(trip_code)
        elif ( ( (trip_count_old +1 ) == cur_trip_count) ):
            if date_old == cur_date:
                df_single_ap.at[idx,'trip_code'] =  str(trip_code)            
            else:
                trip_code += 1
        else:
            trip_code += 1
            df_single_ap.at[idx,'trip_code'] =  str(trip_code)

        trip_count_old = cur_trip_count
        date_old = cur_date

    return df_single_ap 


In [2]:
csv='data/2019-04_2019-08_GyokuSendo.csv'
final_csv='data/2019-04_2019-08_GyokuSendo_staypoints.csv'

df = pd.read_csv(csv,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])
df.rename(columns = {'serial':'ap_id','tlm_datagettime':'timestamp'}, inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df=df.sort_values(by=['timestamp'])

In [3]:
#df = df.query("ap_id=='AP622732'") 

In [4]:
arr_ap_ids = df.ap_id.unique()
#ap_id = arr_ap_ids[0]
#ap_id

arr_trip_summary = []

for ap_id in arr_ap_ids:
    df_single_ap = df.query("ap_id=='"+ap_id+"'").copy()
    df_single_ap = df_single_ap.sort_values(by=['timestamp'])
    arr_trips = df_single_ap.tripcount.unique()
    #trip_count = arr_trips[0]
    #trip_count
    ts_prevs = df_single_ap.timestamp.min()
    lat_prevs = df_single_ap.iloc[0].lat
    lon_prevs = df_single_ap.iloc[0].lon
    
    trip_count_prevs = -9
    for trip_count in arr_trips:
        
        '''
        if (trip_count -1) != trip_count_prevs: # do not process trips which are not consicutive  e.g. trip_count_prevs = 10 and  trip_count= 13
            trip_count_prevs = trip_count
            print(trip_count)
            continue
        '''
         
        # process consicutive trips only e.g. trip_count_prevs = 12 and  trip_count=13
        df_single_trip = df_single_ap.query("tripcount=='"+str(trip_count)+"'").copy()
        df_single_trip = df_single_trip.sort_values(by=['timestamp'])
        max_ts = df_single_trip.timestamp.max()
        min_ts= df_single_trip.timestamp.min()
        
        stay_time =  min_ts - ts_prevs
        trip_time = (max_ts - min_ts).total_seconds()
        
        #if (trip_time < 1800): # ignore if the stay_time is less than 30 minutes
        #    continue
          
        
        lat_min = df_single_trip.iloc[0].lat
        lon_min = df_single_trip.iloc[0].lon
        lat_max = df_single_trip.iloc[len(df_single_trip)-1].lat 
        lon_max = df_single_trip.iloc[len(df_single_trip)-1].lon
        
        avg_lat = (lat_prevs + lat_min)/2
        avg_lon = (lon_prevs + lon_min)/2
        
        lat_prevs = lat_max
        lon_prevs = lon_max
        
        if ( trip_count_prevs  == trip_count-1):
            
            arr_trip_summary.append({
                'ap_id': ap_id,

                'trip_prevs': trip_count_prevs,
                'trip_count': trip_count,
                'ts_prevs': ts_prevs,
                'ts_min': min_ts,
                #'ts_max': max_ts,#
                #'trip_time': trip_time,#
                'stay_time': stay_time.total_seconds(),
                'avg_lon_with_prvs': avg_lon,
                'avg_lat_with_prvs': avg_lat,
                #'lat_min': lat_min,#
                #'lon_min': lon_min,#
                #'lat_max': lat_max,    #        
                #'lon_max': lon_max #
                })
            
        trip_count_prevs = trip_count
        ts_prevs = max_ts
    
trip_df =  pd.DataFrame(arr_trip_summary)      
trip_df['date'] = trip_df['ts_min'].dt.date


In [5]:
#trip_df.query("ap_id=='AR710908'")

In [6]:
len(trip_df)

5749

# merge consicutive trips on same day

In [7]:
arr_ap_ids = trip_df.ap_id.unique()

arr_trip_merged = []
for ap_id in arr_ap_ids:
    df_single_ap = trip_df.query("ap_id=='"+ap_id+"'").copy()
    #display(df_single_ap)
    df_single_ap = assign_trip_code(df_single_ap)
    #display(df_single_ap)
    
    arr_trip_code = df_single_ap.trip_code.unique()

    for trip_code in arr_trip_code:
        df_trip_code = df_single_ap.query("trip_code=='"+str(trip_code)+"'")
        
        arr_trip_merged.append( {
                    'ap_id': ap_id,
                    'date':df_trip_code.date.min(),
                    'trip_prevs': df_trip_code.trip_prevs.min(),
                    'trip_count': df_trip_code.trip_count.max(),
                    'ts_prevs': df_trip_code.ts_prevs.min(),
                    'ts_car_start': df_trip_code.ts_min.max(),
                    'stay_time': df_trip_code.stay_time.sum(),
                    'avg_lon_with_prvs': df_trip_code.avg_lon_with_prvs.mean(),
                    'avg_lat_with_prvs':  df_trip_code.avg_lat_with_prvs.mean(),
                    'trip_code': trip_code
                    } )
            
df_final =  pd.DataFrame(arr_trip_merged)      
df_final.to_csv(final_csv, index=False) 

In [8]:
len(df_final)

4982

# remove entries with stay_time < 1800 seconds

In [9]:
df_final = df_final[df_final['stay_time']>=1800]

In [24]:
print('Final output rows:', len(df_final) )

Final output rows: 4847


In [11]:
df_final.head(10)

,ap_id,date,trip_prevs,trip_count,ts_prevs,ts_car_start,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,trip_code
0,AP521745,2019-04-01,2080.0,2081.0,2019-04-01 08:49:43,2019-04-01 11:14:20,8677.0,127.748598,26.141213,1
1,AP521745,2019-04-04,2101.0,2102.0,2019-04-04 12:15:45,2019-04-04 18:05:37,20992.0,127.747382,26.141103,2
2,AP521745,2019-04-08,2123.0,2124.0,2019-04-08 11:23:41,2019-04-08 13:51:25,8864.0,127.749557,26.141103,3
3,AP521745,2019-04-13,2152.0,2153.0,2019-04-13 10:59:40,2019-04-13 12:58:27,7127.0,127.750141,26.140792,4
4,AP521745,2019-04-25,2269.0,2270.0,2019-04-25 10:32:25,2019-04-25 14:24:14,13909.0,127.748954,26.141355,5
5,AP521745,2019-08-09,2975.0,2976.0,2019-08-09 16:51:42,2019-08-09 18:25:04,5602.0,127.749008,26.141453,6
6,AP521745,2019-08-15,3005.0,3006.0,2019-08-15 12:09:10,2019-08-15 15:22:56,11626.0,127.748747,26.141367,7
7,AP521745,2019-08-21,3046.0,3047.0,2019-08-21 13:11:59,2019-08-21 17:17:55,14756.0,127.746495,26.140277,8
8,AR2F0223,2019-04-01,282.0,283.0,2019-04-01 09:17:39,2019-04-01 11:18:10,7231.0,127.748959,26.141311,1
9,AR2F0223,2019-04-06,339.0,340.0,2019-04-06 08:56:49,2019-04-06 10:51:21,6872.0,127.749011,26.141285,2


# Examine result and inoutdata

In [13]:
trip_df1 = df_final.query("ap_id=='AP622732'")
trip_df1.head(10)

,ap_id,date,trip_prevs,trip_count,ts_prevs,ts_car_start,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,trip_code
410,AP622732,2019-04-02,3266.0,3267.0,2019-04-02 11:49:58,2019-04-02 14:01:57,7919.0,127.749320,26.141297,1
411,AP622732,2019-04-05,3293.0,3296.0,2019-04-05 10:35:17,2019-04-05 13:58:22,12109.0,127.749517,26.141255,2
412,AP622732,2019-04-16,3375.0,3376.0,2019-04-16 14:22:33,2019-04-16 16:07:05,6272.0,127.748925,26.141631,3
413,AP622732,2019-08-03,3985.0,3987.0,2019-08-03 14:35:22,2019-08-03 16:07:31,5497.0,127.746530,26.139835,4
414,AP622732,2019-08-04,4001.0,4002.0,2019-08-04 18:07:37,2019-08-04 19:46:46,5949.0,127.746888,26.138636,5
416,AP622732,2019-08-05,4003.0,4004.0,2019-08-05 09:05:42,2019-08-05 09:49:27,2625.0,127.747461,26.141524,6


In [22]:
df.query("ap_id=='AP622732' and  tripcount >4000")#.query("tripcount >4000")

,ap_id,tripid,tripcount,timestamp,lon,lat
20843,AP622732,2019-08-04 17:06:39,4001,2019-08-04 18:06:44,127.746784,26.143575
20844,AP622732,2019-08-04 17:06:39,4001,2019-08-04 18:07:08,127.747760,26.141236
20845,AP622732,2019-08-04 17:06:39,4001,2019-08-04 18:07:37,127.746837,26.138477
20846,AP622732,2019-08-04 19:28:42,4002,2019-08-04 19:46:46,127.746939,26.138796
20847,AP622732,2019-08-04 19:28:42,4002,2019-08-04 19:47:19,127.747792,26.141402
20848,AP622732,2019-08-04 19:28:42,4002,2019-08-04 19:47:51,127.746586,26.143693
20849,AP622732,2019-08-05 08:49:37,4003,2019-08-05 09:01:04,127.746859,26.138546
20850,AP622732,2019-08-05 08:49:37,4003,2019-08-05 09:01:40,127.747470,26.140937
20851,AP622732,2019-08-05 08:49:37,4003,2019-08-05 09:02:44,127.747464,26.141672
20852,AP622732,2019-08-05 08:49:37,4003,2019-08-05 09:03:26,127.747462,26.141534
